# AD9081 + ZCU102 Example Configuration [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/analogdevicesinc/pyadi-jif/blob/main/examples/ad9081_rxtx_example.ipynb)




This example walks through the process of creating a valid configuration for the AD9081 FMC card and ZCU102 FPGA development kit. It leverages the tools pyadi-jif to create configurations and pyadi-dt to generate template devicetrees.

In [1]:
# Install dependencies
#!pip install --index-url https://test.pypi.org/simple/ pyadi-jif[cplex] > /dev/null

In [2]:
# Determine AD9081+ZCU102 Configuration For RX and TX contrained together

import adijif
from pprint import pprint

vcxo = 100e6

sys = adijif.system("ad9081", "hmc7044", "xilinx", vcxo, solver="CPLEX")
sys.fpga.setup_by_dev_kit_name("zcu102")


# Find matching JESD204 params
params = {'L': 4, 'M': 4, 'F': 2, 'jesd_class': 'jesd204b'}
rx_mode = adijif.utils.get_jesd_mode_from_params(sys.converter.adc, **params)
tx_mode = adijif.utils.get_jesd_mode_from_params(sys.converter.dac, **params)

sys.converter.dac.set_quick_configuration_mode(tx_mode[0]['mode'], "jesd204b")
sys.converter.adc.set_quick_configuration_mode(rx_mode[0]['mode'], "jesd204b")
sys.converter.clocking_option = "integrated_pll"
sys.converter.dac.sample_clock = int(7.2e9) / (12 * 1)
sys.converter.adc.sample_clock = int(3.6e9) / (6 * 1)

# Datapath settings
sys.converter.dac.datapath.cduc_interpolation = 12
sys.converter.adc.datapath.cddc_enabled = [False, False, False, False]
sys.converter.adc.datapath.cddc_decimations = [6, 6, 6, 6]
sys.converter.adc.datapath.cddc_enabled = [True, True, True, True]

# The HDL design wants a core clock and not a device (frame) clock
sys.fpga.requires_core_clock_from_device_clock = True
sys.fpga.force_qpll = {
    sys.converter.adc: True,
    sys.converter.dac: True,
}

print(f'DAC Late rate: {sys.converter.dac.bit_clock}')
print(f'ADC Late rate: {sys.converter.adc.bit_clock}')

cfg = sys.solve()
assert cfg
pprint(cfg)



DAC Late rate: 12000000000.0
ADC Late rate: 12000000000.0
Continued
Continued
{'clock': {'n2': 24,
           'out_dividers': [5, 128, 128, 8, 8, 8, 8],
           'output_clocks': {'AD9081_ref_clk': {'divider': 5,
                                                'rate': 480000000.0},
                             'adc_sysref': {'divider': 128, 'rate': 18750000.0},
                             'dac_sysref': {'divider': 128, 'rate': 18750000.0},
                             'zcu102_adc_device_clk': {'divider': 8,
                                                       'rate': 300000000.0},
                             'zcu102_adc_ref_clk': {'divider': 8,
                                                    'rate': 300000000.0},
                             'zcu102_dac_device_clk': {'divider': 8,
                                                       'rate': 300000000.0},
                             'zcu102_dac_ref_clk': {'divider': 8,
                                                    'ra

# Build HDL

Generate necessary make command for the HDL reference design with the required parameters

In [3]:
tx = sys.converter.dac
rx = sys.converter.adc

# Create make command for JESD
def add_params(rtx, RT):
  return f"{RT}X_LANE_RATE={int(rtx.bit_clock/1e9)} {RT}X_JESD_L={rtx.L} {RT}X_JESD_M={rtx.M} {RT}X_JESD_S={rtx.S}"
make_cmd = f"make JESD_MODE={rx.encoding.upper()} "
make_cmd += add_params(rx, "R")+" "
make_cmd += add_params(tx, "T")
print(make_cmd)

make JESD_MODE=8B10B RX_LANE_RATE=12 RX_JESD_L=4 RX_JESD_M=4 RX_JESD_S=1 TX_LANE_RATE=12 TX_JESD_L=4 TX_JESD_M=4 TX_JESD_S=1


# Create Updated node for AD9081, FPGA, and Clocks

Leverage pyadi-dt to translate configuration to example devicetree.


In [4]:
#!pip install --force-reinstall --no-cache-dir git+https://github.com/analogdevicesinc/pyadi-dt.git  > /dev/null

In [ ]:
import adidt

fmc = adidt.ad9081_fmc()
# Change name for backwards compatibility
cfg['clock']['output_clocks']['adc_fpga_link_out_clk'] = cfg['clock']['output_clocks']['zcu102_adc_device_clk']
cfg['clock']['output_clocks']['dac_fpga_link_out_clk'] = cfg['clock']['output_clocks']['zcu102_dac_device_clk']
cfg['clock']['output_clocks']['adc_fpga_ref_clk'] = cfg['clock']['output_clocks']['zcu102_adc_ref_clk']
cfg['clock']['output_clocks']['dac_fpga_ref_clk'] = cfg['clock']['output_clocks']['zcu102_dac_ref_clk']

clock, adc, dac, fpga = fmc.map_clocks_to_board_layout(cfg)
fn = fmc.gen_dt(clock=clock, adc=adc, dac=dac, fpga=fpga)

# Read in generate DTS file
with open(fn, 'r') as f:
    dts = f.read()

print(dts)

KeyError: 'adc_fpga_ref_clk'